In [13]:
import sys
import os

# 1. Agregamos la carpeta superior al sistema (para encontrar config.py)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# 2. Ahora sí importamos tu configuración
try:
    from config import PATHS
    print("✅ Configuración importada correctamente.")
except ImportError:
    print("❌ No se encontró el archivo 'config.py' en la carpeta superior.")
    # Definimos PATHS dummy para que no rompa si falla el import
    PATHS = {"raw_sla": "."} 

# 3. Resto de librerías
import glob
import re
import datetime
import pandas as pd
import numpy as np

✅ Configuración importada correctamente.


In [14]:

def obtener_rango_fechas(nombre_archivo):
    """
    Extrae fechas usando TU patrón validado: 'd-m-y al d-m-y'
    Retorna: Fecha Inicio, Fecha Fin, Etiqueta (Ej: 'ENE 2026 Q1')
    """
    # 1. Limpieza básica
    nombre_limpio = os.path.basename(nombre_archivo).lower()
    
    # 2. TU REGEX (La que sí funcionaba)
    # Busca algo como: "1-12-2025 al 15-12-2025"
    patron = r"(\d{1,2}-\d{1,2}-\d{4})\s+al\s+(\d{1,2}-\d{1,2}-\d{4})"
    
    match = re.search(patron, nombre_limpio)
    
    if not match:
        return None, None, None

    try:
        str_inicio, str_fin = match.groups()
        
        # 3. Conversión directa (Asumiendo formato dd-mm-yyyy del nombre de archivo)
        f_inicio = datetime.datetime.strptime(str_inicio, "%d-%m-%Y")
        f_fin = datetime.datetime.strptime(str_fin, "%d-%m-%Y")

        # 4. Lógica de Quincena (Q1 vs Q2)
        # Regla: Si la fecha fin es el día 15 (o antes), es la primera quincena.
        #        Si la fecha fin es mayor al 15 (ej. 30 o 31), es la segunda quincena.
        quincena_str = "Q1" if f_fin.day <= 15 else "Q2"
        
        # 5. Nombre del Mes (En Español)
        # Usamos f_fin.month para saber a qué mes pertenece el cierre
        meses = ["", "ENE", "FEB", "MAR", "ABR", "MAY", "JUN", 
                 "JUL", "AGO", "SEP", "OCT", "NOV", "DIC"]
        
        # Construimos la etiqueta: "DIC 2025 Q1"
        nombre_etiqueta = f"{meses[f_fin.month]} {f_fin.year} {quincena_str}"
        
        return f_inicio, f_fin, nombre_etiqueta

    except Exception as e:
        print(f"[red]❌ Error calculando fechas en {nombre_archivo}: {e}[/]")
        return None, None, None

In [15]:
import os
import glob
import re
import datetime
import pandas as pd
import numpy as np

# ==========================================
# 1. FUNCIONES UTILITARIAS
# ==========================================
def obtener_rango_fechas(nombre_archivo):
    try:
        nombre_limpio = os.path.basename(nombre_archivo).lower()
        patron = r"(\d{1,2}-\d{1,2}-\d{4})\s+al\s+(\d{1,2}-\d{1,2}-\d{4})"
        match = re.search(patron, nombre_limpio)
        if not match: return None, None, None

        str_inicio, str_fin = match.groups()
        f_inicio = datetime.datetime.strptime(str_inicio, "%d-%m-%Y")
        f_fin = datetime.datetime.strptime(str_fin, "%d-%m-%Y")

        quincena_str = "Q1" if f_fin.day <= 15 else "Q2"
        meses = ["", "ENE", "FEB", "MAR", "ABR", "MAY", "JUN", 
                 "JUL", "AGO", "SEP", "OCT", "NOV", "DIC"]
        nombre_etiqueta = f"{meses[f_fin.month]} {f_fin.year} {quincena_str}"
        return f_inicio, f_fin, nombre_etiqueta
    except Exception:
        return None, None, None

def limpiar_fechas_mixtas(series):
    """
    Función maestra para columnas que vienen sucias (mezcla de fechas, números y texto).
    """
    # 1. Si ya es fecha (datetime), no tocamos nada
    if pd.api.types.is_datetime64_any_dtype(series):
        return series

    # 2. Convertir a numérico (para seriales Excel)
    # Los seriales válidos están entre 35000 (año 1995) y 60000 (año 2064)
    series_nums = pd.to_numeric(series, errors='coerce')
    mask_es_serial = (series_nums > 35000) & (series_nums < 60000)
    
    # Convertimos los seriales seguros
    fechas_excel = pd.to_datetime(series_nums[mask_es_serial], unit='D', origin='1899-12-30')
    
    # 3. El resto lo tratamos como texto con dayfirst=True
    # Solo procesamos lo que NO era serial
    resto = series[~mask_es_serial].astype(str).str.strip()
    # errors='coerce' pondrá NaT a la basura (teléfonos, textos raros)
    fechas_texto = pd.to_datetime(resto, dayfirst=True, errors='coerce')
    
    # 4. Combinar: Preferencia a los seriales, relleno con texto
    return fechas_excel.combine_first(fechas_texto)

# ==========================================
# 2. CONFIGURACIÓN
# ==========================================

# Asegúrate de que PATHS esté definido
# PATHS = {"raw_sla": "C:/Ruta/..."}

NOC_USERS = [
    "GFARFAN", "JVELASQUEZ", "JOLUGO", "KUSEA", "SLOPEZ",
    "EDESPINOZA", "SANDYJIM", "JOCASTILLO", "JESUSGARCIA",
    "DFUENTES", "JOCANTO", "IXMONTILLA", "OMTRUJILLO",
    "LLZERPA", "LUIJIMENEZ"
]

EXCLUIR_SOLUCIONES = [
    "CAMBIO DE CLAVE", "CLIENTE SOLICITO REEMBOLSO", 
    "LLAMADAS DE AGENDAMIENTO", "ORDEN REPETIDA"
]

COLS_EXTRACCION = [
    "FechaInicio", "FechaFin", "FechaInicioQuincena", "Quincena Evaluada",
    "N° Contrato", "Estatus contrato", "N° Orden", "Estatus_orden",
    "Fecha Creacion", "Fecha Impresion", "Fecha Finalizacion", 
    "Grupo Afinidad", "Detalle Orden", "Franquicia",
    "Grupo Trabajo", "Usuario Emisión", "Usuario Impresión", 
    "Usuario Final", "Solucion Aplicada"
]

ORDEN_FINAL = [
    "Fecha Creacion", "Fecha Impresion", "Fecha Finalizacion",
    "SLA Resolucion", "SLA Despacho", "SLA Impresion",
    "Clasificacion", "N° Orden", "N° Contrato", 
    "Grupo Trabajo", "Usuario Final", "Solucion Aplicada",
    "Estatus_orden", "Detalle Orden", "Grupo Afinidad", 
    "Franquicia", "Usuario Emisión", "Usuario Impresión", "Estatus contrato",
    "Quincena Evaluada", "FechaInicio", "FechaFin"
]

ruta_origen = PATHS.get("raw_sla")
if not ruta_origen or not os.path.exists(ruta_origen):
    print(f"❌ Error: Ruta inválida {ruta_origen}")
else:
    archivos = glob.glob(os.path.join(ruta_origen, "*.xlsx"))
    print(f"📂 Se encontraron {len(archivos)} archivos.")

    dataframes_procesados = []

    # ==========================================
    # 3. PROCESAMIENTO
    # ==========================================

    for archivo in archivos:
        nombre_archivo = os.path.basename(archivo)

        if nombre_archivo.startswith("~$") or "Consolidado" in nombre_archivo:
            continue
        
        fecha_inicio, fecha_fin, quincena_nombre = obtener_rango_fechas(nombre_archivo)
        if not fecha_inicio:
            print(f"⚠️ Saltando {nombre_archivo} (Sin fechas válidas)")
            continue

        try:
            # --- C. LECTURA (CAMBIO IMPORTANTE: Sin dtype=str) ---
            # Al quitar dtype=str, dejamos que 'calamine' detecte las fechas nativas de Excel.
            # Esto evita la ambigüedad 1-11 (Enero vs Noviembre).
            df = pd.read_excel(archivo, engine="calamine")
            
            if df.empty: continue

            # --- D. CORRECCIÓN DE FECHAS INTELIGENTE ---
            cols_fecha = ["Fecha Creacion", "Fecha Emisión", "Fecha Final", 
                          "Fecha Impresion", "Fecha Cierre", "Fecha Finalizacion"]
            
            for col in cols_fecha:
                if col in df.columns:
                    # Usamos la función que respeta lo que ya es fecha y arregla lo demás
                    df[col] = limpiar_fechas_mixtas(df[col])

            # --- E. FILTRO CRUZADO ---
            if "Fecha Finalizacion" not in df.columns:
                print(f"❌ {nombre_archivo} falta Fecha Finalizacion")
                continue

            # Filtramos solo por fecha finalización
            df_filtrado = df.query("@fecha_inicio <= `Fecha Finalizacion` <= @fecha_fin").copy()
            
            if len(df_filtrado) == 0:
                print(f"⚠️ {nombre_archivo}: 0 filas. (Rango: {fecha_inicio.date()} al {fecha_fin.date()})")
                continue

            # --- F. ENRIQUECIMIENTO ---
            df_filtrado = df_filtrado.assign(
                FechaInicio = fecha_inicio,
                FechaFin = fecha_fin,
                FechaInicioQuincena = fecha_inicio,
                Quincena_Evaluada = quincena_nombre
            ).rename(columns={"Quincena_Evaluada": "Quincena Evaluada"})

            # --- G. LIMPIEZA NEGOCIO ---
            df_filtrado = df_filtrado.assign(
                Solucion_Norm = lambda x: x["Solucion Aplicada"].fillna("").astype(str).str.upper(),
                Grupo_Norm    = lambda x: x["Grupo Trabajo"].fillna("").astype(str).str.upper(),
                Detalle_Norm  = lambda x: x["Detalle Orden"].fillna("").astype(str).str.upper(),
                Estatus_Norm  = lambda x: x["Estatus_orden"].fillna("").astype(str).str.upper()
            )
            
            mask_excluir = (
                df_filtrado["Solucion_Norm"].isin(EXCLUIR_SOLUCIONES) |
                df_filtrado["Grupo_Norm"].str.contains("GT API FIBEX", na=False) |
                (df_filtrado["Detalle_Norm"] == "PRUEBA DE INTERNET") |
                df_filtrado["Estatus_Norm"].str.contains("CREACIÓN", na=False)
            )
            
            df_final = df_filtrado[~mask_excluir].reindex(columns=COLS_EXTRACCION)
            dataframes_procesados.append(df_final)
            print(f" ✅ {nombre_archivo} -> {len(df_final)} filas OK")

        except Exception as e:
            print(f" ❌ Error crítico en {nombre_archivo}: {e}")

    # ==========================================
    # 4. CONSOLIDACIÓN
    # ==========================================

    print(f"\nConsolidando {len(dataframes_procesados)} DataFrames...")

    if dataframes_procesados:
        review_r = (
            pd.concat(dataframes_procesados, ignore_index=True)
            .assign(
                Grupo_Norm   = lambda x: x['Grupo Trabajo'].fillna('').astype(str).str.upper(),
                Usuario_Norm = lambda x: x['Usuario Final'].fillna('').astype(str).str.upper()
            )
            .assign(
                Clasificacion = lambda x: np.select(
                    condlist=[
                        x['Usuario_Norm'].isin(NOC_USERS),
                        x['Grupo_Norm'].isin(NOC_USERS),
                        x['Usuario_Norm'].str.contains('NOC', na=False),
                        x['Grupo_Norm'].str.contains('OPERACIONES', na=False)
                    ],
                    choicelist=['NOC', 'NOC', 'NOC', 'OPERACIONES'],
                    default='MESA DE CONTROL'
                )
            )
            .assign(
                SLA_Despacho   = lambda x: x['Fecha Finalizacion'] - x['Fecha Impresion'],
                SLA_Resolucion = lambda x: x['Fecha Finalizacion'] - x['Fecha Creacion'],
                SLA_Impresion  = lambda x: x['Fecha Impresion'] - x['Fecha Creacion']
            )
            .query("SLA_Resolucion.notna()")
            .drop(columns=['Grupo_Norm', 'Usuario_Norm'])
            .rename(columns={
                "SLA_Resolucion": "SLA Resolucion",
                "SLA_Despacho":   "SLA Despacho",
                "SLA_Impresion":  "SLA Impresion"
            })
            .reindex(columns=ORDEN_FINAL)
        )

        print(f"📊 Resultado Final: {len(review_r)} filas consolidadas.")
        display(review_r.head())
        
    else:
        print("⚠️ No se generaron datos.")

📂 Se encontraron 13 archivos.
 ✅ Data - SLA 1-10-2025 al 15-11-2025.xlsx -> 29807 filas OK
 ✅ Data - SLA 1-11-2025 al 15-12-2025.xlsx -> 27646 filas OK
 ✅ Data - SLA 1-12-2025 al 15-1-2026.xlsx -> 28404 filas OK
 ✅ Data - SLA 1-7-2025 al 15-8-2025.xlsx -> 31707 filas OK
 ✅ Data - SLA 1-8-2025 al 15-9-2025.xlsx -> 29699 filas OK
 ✅ Data - SLA 1-9-2025 al 15-10-2025.xlsx -> 29250 filas OK
 ✅ Data - SLA 15-10-2025 al 30-11-2025.xlsx -> 29265 filas OK
 ✅ Data - SLA 15-11-2025 al 31-12-2025.xlsx -> 27892 filas OK
 ✅ Data - SLA 15-7-2025 al 31-8-2025.xlsx -> 31385 filas OK
 ✅ Data - SLA 15-8-2025 al 30-9-2025.xlsx -> 30616 filas OK
 ✅ Data - SLA 15-9-2025 al 31-10-2025.xlsx -> 30805 filas OK
⚠️ Saltando Data - SLA Agosto 2025.xlsx (Sin fechas válidas)

Consolidando 11 DataFrames...
📊 Resultado Final: 326476 filas consolidadas.


,Fecha Creacion,Fecha Impresion,Fecha Finalizacion,SLA Resolucion,SLA Despacho,SLA Impresion,Clasificacion,N° Orden,N° Contrato,Grupo Trabajo,...,Estatus_orden,Detalle Orden,Grupo Afinidad,Franquicia,Usuario Emisión,Usuario Impresión,Estatus contrato,Quincena Evaluada,FechaInicio,FechaFin
0,2025-10-08 08:22:00,2025-10-08 09:03:00,2025-10-08 09:03:00,0 days 00:41:00,0 days 00:00:00,0 days 00:41:00,NOC,V200444,H9131,GERLY FARFAN,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,AIHPXIX,GFARFAN,ACTIVO,NOV 2025 Q1,2025-10-01,2025-11-15
1,2025-11-04 17:40:00,2025-11-04 17:47:00,2025-11-04 17:56:00,0 days 00:16:00,0 days 00:09:00,0 days 00:07:00,MESA DE CONTROL,V15834,H9162,JHOAN ANDRES YAMARTE (NOC),...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,JYAMARTE,ACTIVO,NOV 2025 Q1,2025-10-01,2025-11-15
2,2025-11-02 18:01:00,2025-11-02 18:36:00,2025-11-02 18:36:00,0 days 00:35:00,0 days 00:00:00,0 days 00:35:00,MESA DE CONTROL,V14254,H9162,CARLOS GONZALEZ (NOC),...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,CARLGONZALEZ,ACTIVO,NOV 2025 Q1,2025-10-01,2025-11-15
3,2025-10-12 18:10:00,2025-10-12 18:31:00,2025-10-12 18:31:00,0 days 00:21:00,0 days 00:00:00,0 days 00:21:00,MESA DE CONTROL,V00803,H9162,DIEGO REYES,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,DIREYES,ACTIVO,NOV 2025 Q1,2025-10-01,2025-11-15
4,2025-10-07 21:00:00,2025-10-07 23:00:00,2025-10-07 23:06:00,0 days 02:06:00,0 days 00:06:00,0 days 02:00:00,NOC,V200371,H9162,DENNY FUENTE,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,DFUENTES,ACTIVO,NOV 2025 Q1,2025-10-01,2025-11-15


In [16]:
review_r = review_r[review_r['Fecha Creacion'] >= review_r['FechaInicio']]
dec = review_r[review_r.FechaInicio == pd.Timestamp('2025-11-01')]

display(dec)

,Fecha Creacion,Fecha Impresion,Fecha Finalizacion,SLA Resolucion,SLA Despacho,SLA Impresion,Clasificacion,N° Orden,N° Contrato,Grupo Trabajo,...,Estatus_orden,Detalle Orden,Grupo Afinidad,Franquicia,Usuario Emisión,Usuario Impresión,Estatus contrato,Quincena Evaluada,FechaInicio,FechaFin
29807,2025-11-01 05:30:00,2025-11-01 05:39:00,2025-11-01 05:41:00,0 days 00:11:00,0 days 00:02:00,0 days 00:09:00,MESA DE CONTROL,V13367,V110203,LUIS JOAQUIN VILLEGAS,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,LJVILLEGAS,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15
29808,2025-11-01 07:10:00,2025-11-01 07:21:00,2025-11-03 10:49:00,2 days 03:39:00,2 days 03:28:00,0 days 00:11:00,OPERACIONES,A07218,A6827,OPERACIONES ARAGUA,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX ARAGUA,APICONEXVE,WILLMARTINEZ,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15
29809,2025-11-01 07:17:00,2025-11-01 07:32:00,2025-11-03 13:48:00,2 days 06:31:00,2 days 06:16:00,0 days 00:15:00,MESA DE CONTROL,V13380,V25679,SOLUCIONES TECNO TYC 2025,...,FINALIZADA,FALLA LOS,HOGAR,FIBEX VALENCIA,APICONEXVE,KUSEA,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15
29810,2025-11-01 07:42:00,2025-11-01 07:43:00,2025-11-01 07:43:00,0 days 00:01:00,0 days 00:00:00,0 days 00:01:00,MESA DE CONTROL,V13389,V68853,CARLOS GONZALEZ (NOC),...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,CARLGONZALEZ,CARLGONZALEZ,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15
29811,2025-11-01 07:00:00,2025-11-01 07:35:00,2025-11-01 07:43:00,0 days 00:43:00,0 days 00:08:00,0 days 00:35:00,MESA DE CONTROL,V13372,V110203,WILLCLAR MARTINEZ,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,WILLMARTINEZ,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57448,2025-12-13 08:00:00,2025-12-13 08:48:00,2025-12-13 08:48:00,0 days 00:48:00,0 days 00:00:00,0 days 00:48:00,NOC,V334819,V113172,SANDY JOSE JIMENEZ BETANCOURT,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,SANDYJIM,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15
57449,2025-12-13 08:00:00,2025-12-13 08:47:00,2025-12-13 08:52:00,0 days 00:52:00,0 days 00:05:00,0 days 00:47:00,MESA DE CONTROL,MGTA180457,MGTA6616,WILLCLAR MARTINEZ,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX MARGARITA,APICONEXVE,WILLMARTINEZ,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15
57450,2025-12-13 08:00:00,2025-12-13 08:53:00,2025-12-13 08:59:00,0 days 00:59:00,0 days 00:06:00,0 days 00:53:00,MESA DE CONTROL,A67841,A59583,WILLCLAR MARTINEZ,...,FINALIZADA,CAIDAS SEGUIDAS DE INTERNET,HOGAR,FIBEX ARAGUA,APICONEXVE,WILLMARTINEZ,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15
57451,2025-12-13 08:03:00,2025-12-13 08:59:00,2025-12-13 08:59:00,0 days 00:56:00,0 days 00:00:00,0 days 00:56:00,NOC,SC182892,SC12660,SANDY JOSE JIMENEZ BETANCOURT,...,FINALIZADA,CAIDAS SEGUIDAS DE INTERNET,HOGAR,FIBEX SAN CARLOS,MLUCENA,SANDYJIM,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15


In [26]:
import pandas as pd
import numpy as np

ruta=r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\SLA_Gold.parquet"
sla = pd.read_parquet(ruta)

# Asegurar que la columna de fecha sea datetime (evita comparaciones con strings)
sla["Fecha Finalizacion"] = pd.to_datetime(sla["Fecha Finalizacion"], errors="coerce")

decem = sla[sla["Fecha Finalizacion"] >= pd.Timestamp('2026-01-01')]

# Calcular medias solo para columnas numéricas y de tipo timedeltas (SLA)
num_mean = decem.select_dtypes(include=[np.number]).mean()
td_mean = decem.select_dtypes(include=['timedelta64[ns]']).mean()
mean_sla = pd.concat([num_mean, td_mean])

print("Mean SLA:")
print(mean_sla)

Mean SLA:
SLA Resolucion   0 days 18:05:58.782150899
SLA Despacho     0 days 15:58:04.275792077
SLA Impresion    0 days 02:07:54.506358821
SLA Resolucion   0 days 18:05:58.782150899
SLA Despacho     0 days 15:58:04.275792077
SLA Impresion    0 days 02:07:54.506358821
dtype: timedelta64[ns]


In [46]:
import pandas as pd

ruta = r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\Afluencia_Gold.parquet"
sla = pd.read_parquet(ruta)
display(sla)





,N° Abonado,Documento,Estatus,Fecha,Vendedor,Grupo Afinidad,Nombre Franquicia,Ciudad,Tipo de afluencia,Oficina,Clasificacion,Hora,Mes,Estado_Sede,Tipo_Sede
0,A61776,12612549,ACTIVO,2025-07-02,YOLIANA CAROLINA PAEZ OFIC ARAGUA,HOGAR,FIBEX ARAGUA,PALO NEGRO,Ventas,ARAGUA,None,08:04 AM,julio,ARAGUA,ALIADO / AGENTE
1,A61783,4224514,ACTIVO,2025-07-02,YOLIANA CAROLINA PAEZ OFIC ARAGUA,HOGAR,FIBEX ARAGUA,PALO NEGRO,Ventas,ARAGUA,None,09:09 AM,julio,ARAGUA,ALIADO / AGENTE
2,A61791,30923483,ACTIVO,2025-07-02,FRANKELLY GABRIELA GARCIA OFIC ARAGUA,HOGAR,FIBEX ARAGUA,MARACAY,Ventas,ARAGUA,None,10:02 AM,julio,ARAGUA,ALIADO / AGENTE
3,A61800,5577594,CORTADO,2025-07-02,FRANKELLY GABRIELA GARCIA OFIC ARAGUA,HOGAR,FIBEX ARAGUA,EL LIMóN,Ventas,ARAGUA,None,11:04 AM,julio,ARAGUA,ALIADO / AGENTE
4,A61820,16551410,CORTADO,2025-07-02,FRANKELLY GABRIELA GARCIA OFIC ARAGUA,HOGAR,FIBEX ARAGUA,MARACAY,Ventas,ARAGUA,None,12:24 PM,julio,ARAGUA,ALIADO / AGENTE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220382,A75747,None,POR INSTALAR,2026-01-31,ANGELICA ANGULO OFIC ARAGUA,HOGAR,FIBEX ARAGUA,EL LIMóN,RECAUDACIÓN,ARAGUA,ALIADOS Y DESARROLLO,None,enero,ARAGUA,ALIADO / AGENTE
1220383,A75770,None,SIN FACTIBILIDAD,2026-01-31,JENA MARIAN FIGUEROA ATC AGENTE AUTORIZADO IG ...,HOGAR,FIBEX ARAGUA,TURMERO,RECAUDACIÓN,ARAGUA,ALIADOS Y DESARROLLO,None,enero,ARAGUA,ALIADO / AGENTE
1220384,MI14243,None,POR INSTALAR,2026-01-31,HILDA GERMANIA NAVARRO PEREZ ADM DE VENTAS CAR...,HOGAR,FIBEX MIRANDA,GUARENAS,RECAUDACIÓN,EL PARAISO (LAS FUENTES),OFICINAS PROPIAS,None,enero,CAPITAL,PROPIA
1220385,MI14249,None,POR INSTALAR,2026-01-31,HILDA GERMANIA NAVARRO PEREZ ADM DE VENTAS CAR...,HOGAR,FIBEX MIRANDA,GUARENAS,RECAUDACIÓN,EL PARAISO (LAS FUENTES),OFICINAS PROPIAS,None,enero,CAPITAL,PROPIA


In [55]:
import pandas as pd

ruta = r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\Stock_Abonados_Gold_Resumen.parquet"
idf = pd.read_parquet(ruta)
display(idf)




,Quincena Evaluada,Franquicia,Total_Abonados,Fecha_Corte
0,AGO 2025 Q1,FIBER HOGAR - MATURIN,0,2025-08-15
1,AGO 2025 Q1,FIBEX ANZOATEGUI,0,2025-08-15
2,AGO 2025 Q1,FIBEX ARAGUA,0,2025-08-15
3,AGO 2025 Q1,FIBEX ARAGUA - MATURIN,0,2025-08-15
4,AGO 2025 Q1,FIBEX BARINAS,0,2025-08-15
...,...,...,...,...
469,SEP 2025 Q2,FIBEX VILLA DE CURA,0,2025-09-30
470,SEP 2025 Q2,FIBEX WAVE,0,2025-09-30
471,SEP 2025 Q2,FIBEX YAGUA,0,2025-09-30
472,SEP 2025 Q2,FIBEX YARITAGUA,0,2025-09-30


In [52]:
import pandas as pd
import os

df = pd.read_parquet(r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\SLA_Gold.parquet")
# 1. Agrupamos por Quincena y Clasificación
df_check = df.groupby(['Quincena Evaluada', 'Clasificacion']).agg(
    Total_Minutos=('SLA Resolucion Min', 'sum'),  # Suma total del tiempo
    Total_Tickets=('Total_Ordenes', 'sum')         # Conteo de tickets ÚNICOS
).reset_index()

# 2. Calculamos el Promedio Real (Ponderado)
df_check['Promedio_SLA_Min'] = df_check['Total_Minutos'] / df_check['Total_Tickets']

# Opcional: Ver en Horas
df_check['Promedio_SLA_Hrs'] = df_check['Promedio_SLA_Min'] / 60

# 3. Mostrar resultado
print(df_check.sort_values(by=['Quincena Evaluada', 'Clasificacion']))
display(df)


   Quincena Evaluada    Clasificacion  Total_Minutos  Total_Tickets  \
0        AGO 2025 Q1  MESA DE CONTROL     50307915.0          24224   
1        AGO 2025 Q1              NOC      1026017.0           7925   
2        AGO 2025 Q1      OPERACIONES      4051352.0            225   
3        AGO 2025 Q2  MESA DE CONTROL     52155101.0          24339   
4        AGO 2025 Q2              NOC      1008762.0           7142   
5        AGO 2025 Q2      OPERACIONES      4036151.0            211   
6        DIC 2025 Q1  MESA DE CONTROL     30138287.0          22955   
7        DIC 2025 Q1              NOC       589990.0           4311   
8        DIC 2025 Q1      OPERACIONES      4476011.0           1600   
9        DIC 2025 Q2  MESA DE CONTROL     24603701.0          22155   
10       DIC 2025 Q2              NOC       355111.0           4098   
11       DIC 2025 Q2      OPERACIONES      3429748.0           1593   
12       ENE 2026 Q1  MESA DE CONTROL     31156780.0          23857   
13    

,Quincena Evaluada,Franquicia,Clasificacion,Fecha_Creacion_Date,Total_Ordenes,SLA Resolucion Min,SLA Despacho Min,SLA Impresion Min
0,AGO 2025 Q1,FIBER HOGAR - MATURIN,MESA DE CONTROL,2025-07-01,16,12894.0,12577.0,317.0
1,AGO 2025 Q1,FIBER HOGAR - MATURIN,MESA DE CONTROL,2025-07-02,13,3196.0,3117.0,79.0
2,AGO 2025 Q1,FIBER HOGAR - MATURIN,MESA DE CONTROL,2025-07-03,13,13662.0,13378.0,284.0
3,AGO 2025 Q1,FIBER HOGAR - MATURIN,MESA DE CONTROL,2025-07-04,10,29507.0,29363.0,144.0
4,AGO 2025 Q1,FIBER HOGAR - MATURIN,MESA DE CONTROL,2025-07-05,11,41531.0,41381.0,150.0
...,...,...,...,...,...,...,...,...
30281,SEP 2025 Q2,FIBEX YARITAGUA,NOC,2025-09-23,1,208.0,0.0,208.0
30282,SEP 2025 Q2,FIBEX YARITAGUA,NOC,2025-09-24,1,63.0,1.0,62.0
30283,SEP 2025 Q2,FIBEX YARITAGUA,NOC,2025-09-25,1,153.0,0.0,153.0
30284,SEP 2025 Q2,FIBEX YARITAGUA,NOC,2025-09-29,1,33.0,3.0,30.0


In [1]:
import pandas as pd
from utils import *
from config import *
import os

df_abo = pd.read_parquet(r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\IdF_Gold.parquet")
filter = df_abo[(df_abo["Quincena Evaluada"] == "NOV 2025 Q1") & (df_abo["Fecha_Creacion_Date"]).str.contains("2025-11")]
franquicia = filter.groupby(["Quincena Evaluada"], as_index=False).sum("Total_Fallas")
display(franquicia)

,Quincena Evaluada,Total_Fallas
0,NOV 2025 Q1,9193


958


In [ ]:
import random
import pandas
letters = [
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o',
    'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D',
    'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S',
    'T', 'U', 'V', 'W', 'X', 'Y', 'Z'
]
numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
symbols = ['!', '#', '$', '%', '&', '(', ')', '*', '+']

pasword = []
print("Welcome to your PyPassword generator")

Question1=int(input("How many letters do you want in your password?\n "))
Question2=int(input("How many number do you want?\n "))
Question3=int(input("How many symbols do you want?\n "))
for y in range:
    for i in range(Question1):
        ramdon_letter=random.choice(letters)
        pasword.append(ramdon_letter)
        #print(pasword)


    for i in range(Question2):
        ramdon_number=random.choice(numbers)
        pasword.append(ramdon_number)


    for i in range(Question3):
        ramdon_symbol=random.choice(symbols)
        pasword.append(ramdon_symbol)
    resultado = "".join(map(str,pasword))
print(resultado)


Welcome to your PyPassword generator
mUV71965%)()#&


In [20]:
import pandas as pd

df_abo = pd.read_parquet(r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\Ventas_Listado_Gold.parquet")
filter = ["Canal","Estatus","Fecha Contrato"]
df_abo = df_abo[df_abo["Canal"].str.contains("OFICINA COMERCIAL", case=False, na=False)]
df_abo = df_abo.groupby(filter, as_index=False).size()
display(df_abo)


,Canal,Estatus,Fecha Contrato,size
0,OFICINA COMERCIAL,ACTIVO,2025-07-01,45
1,OFICINA COMERCIAL,ACTIVO,2025-07-02,57
2,OFICINA COMERCIAL,ACTIVO,2025-07-03,50
3,OFICINA COMERCIAL,ACTIVO,2025-07-04,48
4,OFICINA COMERCIAL,ACTIVO,2025-07-05,30
...,...,...,...,...
423,OFICINA COMERCIAL,SIN FACTIBILIDAD,2026-02-16,1
424,OFICINA COMERCIAL,SIN FACTIBILIDAD,2026-02-17,1
425,OFICINA COMERCIAL,SIN FACTIBILIDAD,2026-02-18,1
426,OFICINA COMERCIAL,SIN FACTIBILIDAD,2026-02-19,3
